In [ ]:
import numpy as np


In [ ]:
datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')

In [ ]:
print(datain)
print(datain.shape)   # Useful if it’s an array
print(type(datain)) 

In [ ]:
print(dataout)
print(dataout.shape)   # Useful if it’s an array
print(type(dataout)) 

In [ ]:
import numpy as np
from scipy.optimize import minimize, differential_evolution
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

# -------------------------
# Put your data here
# -------------------------
X_init = datain
y_init = dataout
# -------------------------
# Transform / settings
# -------------------------
bounds = np.array([(0.0, 1.0)] * X_init.shape[1])  # assume normalized [0,1] per dim
dim = X_init.shape[1]
y_best = y_init.max()

# -------------------------
# Fit GP surrogate
# -------------------------
kernel = Matern(length_scale=np.ones(dim),
                length_scale_bounds=(1e-2, 1e2),
                nu=2.5)
# alpha = noise variance. If outputs are noisy, increase alpha (e.g. 1e-4 or tuned)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True, n_restarts_optimizer=5, random_state=0)
gp.fit(X_init, y_init)

# -------------------------
# Expected Improvement (EI)
# -------------------------
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.atleast_2d(x)
    mu, sigma = gp.predict(x, return_std=True)
    sigma = np.maximum(sigma, 1e-12)
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return ei  # note: maximize this

# Negative EI for minimizers
def neg_ei(x, gp, y_best, xi=0.01):
    return -expected_improvement(x.reshape(1, -1), gp, y_best, xi=xi)[0]

# -------------------------
# Acquisition optimization: many random starts + DE fallback
# -------------------------
def propose_location(gp, y_best, bounds, n_restarts=40):
    dim = bounds.shape[0]
    best_x = None
    best_val = 1e20

    # Random-start L-BFGS-B
    for _ in range(n_restarts):
        x0 = np.random.uniform(bounds[:,0], bounds[:,1])
        res = minimize(fun=neg_ei,
                       x0=x0,
                       args=(gp, y_best),
                       bounds=bounds,
                       method='L-BFGS-B',
                       options={'maxiter':200})
        if res.fun < best_val:
            best_val = res.fun
            best_x = res.x

    # Try differential evolution as a global check (optional, slower)
    try:
        de_res = differential_evolution(lambda x: neg_ei(x, gp, y_best),
                                       bounds=bounds.tolist(),
                                       maxiter=200, polish=True, seed=0)
        if de_res.fun < best_val:
            best_val = de_res.fun
            best_x = de_res.x
    except Exception:
        pass

    # clamp to bounds and return
    best_x = np.clip(best_x, bounds[:,0], bounds[:,1])
    return best_x

# -------------------------
# If you need a small batch (greedy sequential EI)
# -------------------------
def propose_batch(gp, y_best, bounds, batch_size=3):
    batch = []
    gp_copy = gp
    X_aug = X_init.copy()
    y_aug = y_init.copy()
    for i in range(batch_size):
        x_next = propose_location(gp_copy, y_aug.max(), bounds)
        # "Fake" evaluation: use GP predicted mean (Kriging believer)
        y_fake = gp_copy.predict(x_next.reshape(1, -1))[0]
        # Append to augmented set and refit GP (greedy sequential)
        X_aug = np.vstack([X_aug, x_next])
        y_aug = np.hstack([y_aug, y_fake])
        gp_copy = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
        gp_copy.fit(X_aug, y_aug)
        batch.append(x_next)
    return np.array(batch)

# -------------------------
# Run proposal
# -------------------------
x_next = propose_location(gp, y_best, np.array(bounds))
print("Suggested next point (continuous):", x_next)

# If some dims are categorical/encoded, round or map them here, e.g.:
# x_next[cat_index] = int(np.round(x_next[cat_index] * (num_categories-1)))
#
# Example: If dim 6 encodes optimizer 0..3:
# x_next[6] = int(np.round(x_next[6] * 3))

# -------------------------
# (Optional) propose a small batch
# -------------------------
batch = propose_batch(gp, y_best, np.array(bounds), batch_size=3)
print("Suggested batch of 3 points (continuous):\n", batch)


In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- Initial data ---
#X_init = np.array([...])  # 30x6 array
#y_init = np.array([...])  # 30 outputs
X_init = datain
y_init = dataout

y_best = y_init.max()  # best performance so far

# --- Fit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_init, y_init)

# --- Expected Improvement acquisition function ---
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = mu[0], sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative for minimization in scipy

# --- Optimize acquisition function ---
bounds = [(0,1)]*6
best_x = None
best_ei = float('inf')

for _ in range(20):  # multiple random starts
    x0 = np.random.rand(6)
    res = minimize(lambda x: expected_improvement(x, gp, y_best),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

x_next = best_x
print("Next hyperparameter set to try:", x_next)


# Week 2

In [ ]:
import numpy as np
from scipy.optimize import minimize, differential_evolution
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')
# -------------------------
# Existing data (your previous iteration)
# -------------------------
X_init = datain
y_init = dataout

# Latest evaluated point (the one that gave 9.68029...)
X_new = np.array([[0.203499, 0.167999, 0.195105, 0.035878, 0.999999, 0.999999, 0.224367, 0.498362]])
y_new = np.array([9.6802928063541])

# -------------------------
# Step 1: Append new data
# -------------------------
X_all = np.vstack([X_init, X_new])
y_all = np.concatenate([y_init, y_new])

# -------------------------
# Step 2: Fit updated GP surrogate
# -------------------------
bounds = np.array([(0.0, 1.0)] * X_all.shape[1])
dim = X_all.shape[1]

kernel = Matern(length_scale=np.ones(dim),
                length_scale_bounds=(1e-2, 1e2),
                nu=2.5)

# Lower alpha for smooth deterministic behavior
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6,
                              normalize_y=True, n_restarts_optimizer=5,
                              random_state=0)
gp.fit(X_all, y_all)

# -------------------------
# Step 3: Expected Improvement (EI) — exploitative mode
# -------------------------
def expected_improvement(x, gp, y_best, xi=0.001):  # smaller xi = exploit
    x = np.atleast_2d(x)
    mu, sigma = gp.predict(x, return_std=True)
    sigma = np.maximum(sigma, 1e-12)
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return ei

def neg_ei(x, gp, y_best, xi=0.001):
    return -expected_improvement(x.reshape(1, -1), gp, y_best, xi=xi)[0]

# -------------------------
# Step 4: Optimize acquisition to find next candidate
# -------------------------
def propose_location(gp, y_best, bounds, n_restarts=40):
    best_x, best_val = None, 1e20
    for _ in range(n_restarts):
        x0 = np.random.uniform(bounds[:,0], bounds[:,1])
        res = minimize(fun=neg_ei,
                       x0=x0,
                       args=(gp, y_best, 0.001),
                       bounds=bounds,
                       method='L-BFGS-B',
                       options={'maxiter':200})
        if res.fun < best_val:
            best_val, best_x = res.fun, res.x

    # Optional global search (safety)
    try:
        de_res = differential_evolution(lambda x: neg_ei(x, gp, y_best, 0.001),
                                        bounds=bounds.tolist(),
                                        maxiter=200, polish=True, seed=0)
        if de_res.fun < best_val:
            best_val, best_x = de_res.fun, de_res.x
    except Exception:
        pass

    return np.clip(best_x, bounds[:,0], bounds[:,1])

# Compute updated best observed value
y_best = y_all.max()

# Get next suggested point
x_next = propose_location(gp, y_best, bounds)

print("Next suggested point (exploit mode):", x_next)


# Week 3

In [ ]:
import numpy as np
from scipy.optimize import minimize, differential_evolution
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')
# -------------------------
# Existing data (your previous iteration)
# -------------------------
X_init = datain
y_init = dataout

In [ ]:
# Latest evaluated point (the one that gave 9.68029...) from Week 1
X_new = np.array([[0.203499, 0.167999, 0.195105, 0.035878, 0.999999, 0.999999, 0.224367, 0.498362]])
y_new = np.array([9.6802928063541])

# -------------------------
# Step 1: Append new data
# -------------------------
X_all = np.vstack([X_init, X_new])
y_all = np.concatenate([y_init, y_new])

In [ ]:
# Latest evaluated point (the one that gave 9.62007...) from Week 2
X_new = np.array([[0.172222, 0.268299, 0.139005, 0.307855, 0.999999, 0.999999, 0.348896, 0.999999]])
y_new = np.array([9.6200730832974])

# -------------------------
# Step 1: Append new data
# -------------------------
X_all = np.vstack([X_init, X_new])
y_all = np.concatenate([y_init, y_new])

In [ ]:
print(X_all)
print(X_all.shape)   # Useful if it’s an array
print(type(X_all)) 
print("-----------------------------------------------")
print(y_all)
print(y_all.shape)   # Useful if it’s an array
print(type(y_all)) 

In [ ]:
# -------------------------
# Step 2: Fit updated GP surrogate
# -------------------------
bounds = np.array([(0.0, 1.0)] * X_all.shape[1])
dim = X_all.shape[1]

kernel = Matern(length_scale=np.ones(dim),
                length_scale_bounds=(1e-2, 1e2),
                nu=2.5)

# Lower alpha for smooth deterministic behavior
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6,
                              normalize_y=True, n_restarts_optimizer=5,
                              random_state=0)
gp.fit(X_all, y_all)

# -------------------------
# Step 3: Expected Improvement (EI) — exploitative mode
# -------------------------
def expected_improvement(x, gp, y_best, xi=0.001):  # smaller xi = exploit
    x = np.atleast_2d(x)
    mu, sigma = gp.predict(x, return_std=True)
    sigma = np.maximum(sigma, 1e-12)
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return ei

def neg_ei(x, gp, y_best, xi=0.001):
    return -expected_improvement(x.reshape(1, -1), gp, y_best, xi=xi)[0]

# -------------------------
# Step 4: Optimize acquisition to find next candidate
# -------------------------
def propose_location(gp, y_best, bounds, n_restarts=40):
    best_x, best_val = None, 1e20
    for _ in range(n_restarts):
        x0 = np.random.uniform(bounds[:,0], bounds[:,1])
        res = minimize(fun=neg_ei,
                       x0=x0,
                       args=(gp, y_best, 0.001),
                       bounds=bounds,
                       method='L-BFGS-B',
                       options={'maxiter':200})
        if res.fun < best_val:
            best_val, best_x = res.fun, res.x

    # Optional global search (safety)
    try:
        de_res = differential_evolution(lambda x: neg_ei(x, gp, y_best, 0.001),
                                        bounds=bounds.tolist(),
                                        maxiter=200, polish=True, seed=0)
        if de_res.fun < best_val:
            best_val, best_x = de_res.fun, de_res.x
    except Exception:
        pass

    return np.clip(best_x, bounds[:,0], bounds[:,1])

# Compute updated best observed value
y_best = y_all.max()

# Get next suggested point
x_next = propose_location(gp, y_best, bounds)

print("Next suggested point (exploit mode):", x_next)

x_next_6dp = np.round(x_next, 6)
x_next_6dp
print("Rounded:", x_next_6dp)


# Slight tweak to slightly shrink the search bounds to avoid exact 0 or 1 values,
# without adding any penalties or acquisition modifications.

In [ ]:
# updated propose_location() function with the soft clamping fix integrated.
# This will stop 0.0 and 1.0 values, while leaving everything else about your Bayesian optimization loop unchanged.

In [ ]:
# -------------------------
# Step 4: Optimize EI to get next candidate
# -------------------------
def propose_location(gp, y_best, bounds, n_restarts=40, xi=0.001):
    dim = bounds.shape[0]
    best_x = None
    best_val = 1e20

    # Random-start L-BFGS-B local optimization
    for _ in range(n_restarts):
        x0 = np.random.uniform(bounds[:, 0], bounds[:, 1])
        res = minimize(
            fun=neg_ei,
            x0=x0,
            args=(gp, y_best, xi),
            bounds=bounds,
            method='L-BFGS-B',
            options={'maxiter': 200}
        )
        if res.fun < best_val:
            best_val = res.fun
            best_x = res.x

    # Global backup using Differential Evolution
    try:
        de_res = differential_evolution(
            lambda x: neg_ei(x, gp, y_best, xi),
            bounds=bounds.tolist(),
            maxiter=200,
            polish=True,
            seed=0
        )
        if de_res.fun < best_val:
            best_val = de_res.fun
            best_x = de_res.x
    except Exception:
        pass

    # --------------------------------------
    # Soft clamp to avoid hitting exact edges
    # --------------------------------------
    epsilon = 0.01  # safety margin
    best_x = np.clip(best_x, epsilon, 1 - epsilon)

    return best_x


In [ ]:
# -------------------------
# Step 2: Fit updated GP surrogate
# -------------------------
bounds = np.array([(0.0, 1.0)] * X_all.shape[1])
dim = X_all.shape[1]

kernel = Matern(length_scale=np.ones(dim),
                length_scale_bounds=(1e-2, 1e2),
                nu=2.5)

# Lower alpha for smooth deterministic behavior
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6,
                              normalize_y=True, n_restarts_optimizer=5,
                              random_state=0)
gp.fit(X_all, y_all)

# -------------------------
# Step 3: Expected Improvement (EI) — exploitative mode
# -------------------------
def expected_improvement(x, gp, y_best, xi=0.001):  # smaller xi = exploit
    x = np.atleast_2d(x)
    mu, sigma = gp.predict(x, return_std=True)
    sigma = np.maximum(sigma, 1e-12)
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return ei

def neg_ei(x, gp, y_best, xi=0.001):
    return -expected_improvement(x.reshape(1, -1), gp, y_best, xi=xi)[0]

# -------------------------
# Step 4: Optimize acquisition to find next candidate
# -------------------------
def propose_location(gp, y_best, bounds, n_restarts=40):
    best_x, best_val = None, 1e20
    for _ in range(n_restarts):
        x0 = np.random.uniform(bounds[:,0], bounds[:,1])
        res = minimize(fun=neg_ei,
                       x0=x0,
                       args=(gp, y_best, 0.001),
                       bounds=bounds,
                       method='L-BFGS-B',
                       options={'maxiter':200})
        if res.fun < best_val:
            best_val, best_x = res.fun, res.x

    # Optional global search (safety)
    try:
        de_res = differential_evolution(lambda x: neg_ei(x, gp, y_best, 0.001),
                                        bounds=bounds.tolist(),
                                        maxiter=200, polish=True, seed=0)
        if de_res.fun < best_val:
            best_val, best_x = de_res.fun, de_res.x
    except Exception:
        pass

    return np.clip(best_x, bounds[:,0], bounds[:,1])

# Compute updated best observed value
y_best = y_all.max()

# Get next suggested point
x_next = propose_location(gp, y_best, bounds)

print("Next suggested point (exploit mode):", x_next)

x_next_6dp = np.round(x_next, 6)
x_next_6dp
print("Rounded:", x_next_6dp)


In [ ]:
x_next_6dp = np.round(x_next, 6)
x_next_6dp
print("Rounded to:", x_next_6dp)

# Week 4

In [ ]:
#Note: ran Week 3 with missing data accidentally (41 inputs not 42)


Previous high outputs: 9.6803, 9.6201
This new point improves the current best: y_best = 9.7668
The point is near previous high regions, especially in dimensions 4, 5, and 8 which are now close to the edges (0.01 / 0.99 after clamping).

In [ ]:
import numpy as np
from scipy.optimize import minimize, differential_evolution
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')
# -------------------------
# Existing data (your previous iteration)
# -------------------------
X_init = datain
y_init = dataout

In [ ]:
# Latest evaluated point (the one that gave 9.68029...) from Week 1
X_new = np.array([[0.203499, 0.167999, 0.195105, 0.035878, 0.999999, 0.999999, 0.224367, 0.498362]])
y_new = np.array([9.6802928063541])

# -------------------------
# Step 1: Append new data
# -------------------------
X_all = np.vstack([X_init, X_new])
y_all = np.concatenate([y_init, y_new])

In [ ]:
# Latest evaluated point (the one that gave 9.62007...) from Week 2
X_new = np.array([[0.172222, 0.268299, 0.139005, 0.307855, 0.999999, 0.999999, 0.348896, 0.999999]])
y_new = np.array([9.6200730832974])

# -------------------------
# Step 1: Append new data
# -------------------------
X_all = np.vstack([X_all, X_new])
y_all = np.concatenate([y_all, y_new])

In [ ]:
# Latest evaluated point (the one that gave 9.766762...) from Week 3
X_new = np.array([[0.251068, 0.078758, 0.302102, 0.010000, 0.990000, 0.357561, 0.178631, 0.010000]])
y_new = np.array([9.766762063933])

# -------------------------
# Step 1: Append new data
# -------------------------
X_all = np.vstack([X_all, X_new])
y_all = np.concatenate([y_all, y_new])

In [ ]:
print(X_all)
print(X_all.shape)   # Useful if it’s an array
print(type(X_all)) 
print("-----------------------------------------------")
print(y_all)
print(y_all.shape)   # Useful if it’s an array
print(type(y_all)) 

# Summary
The new output confirms a rising trend toward a local maximum.
Strategy: continue local exploitation near the current best.
Maintain safety margins (clamped bounds) and low xi.
Each iteration: append new (X, y), refit GP, maximize EI → next point.

In [ ]:
# -------------------------
# Step 4: Optimize EI to get next candidate
# -------------------------
def propose_location(gp, y_best, bounds, n_restarts=40, xi=0.001):
    dim = bounds.shape[0]
    best_x = None
    best_val = 1e20

    # Random-start L-BFGS-B local optimization
    for _ in range(n_restarts):
        x0 = np.random.uniform(bounds[:, 0], bounds[:, 1])
        res = minimize(
            fun=neg_ei,
            x0=x0,
            args=(gp, y_best, xi),
            bounds=bounds,
            method='L-BFGS-B',
            options={'maxiter': 200}
        )
        if res.fun < best_val:
            best_val = res.fun
            best_x = res.x

    # Global backup using Differential Evolution
    try:
        de_res = differential_evolution(
            lambda x: neg_ei(x, gp, y_best, xi),
            bounds=bounds.tolist(),
            maxiter=200,
            polish=True,
            seed=0
        )
        if de_res.fun < best_val:
            best_val = de_res.fun
            best_x = de_res.x
    except Exception:
        pass

    # --------------------------------------
    # Soft clamp to avoid hitting exact edges
    # --------------------------------------
    epsilon = 0.01  # safety margin
    best_x = np.clip(best_x, epsilon, 1 - epsilon)

    return best_x


In [ ]:
# -------------------------
# Step 2: Fit Gaussian Process
# -------------------------
bounds = np.array([(0.0, 1.0)] * X_all.shape[1])
dim = X_all.shape[1]

kernel = Matern(length_scale=np.ones(dim), 
                length_scale_bounds=(1e-2, 1e2), 
                nu=2.5)

gp = GaussianProcessRegressor(kernel=kernel,
                              alpha=1e-6,
                              normalize_y=True,
                              n_restarts_optimizer=5,
                              random_state=0)
gp.fit(X_all, y_all)

# -------------------------
# Step 3: Expected Improvement (EI)
# -------------------------
def expected_improvement(x, gp, y_best, xi=0.001):
    x = np.atleast_2d(x)
    mu, sigma = gp.predict(x, return_std=True)
    sigma = np.maximum(sigma, 1e-12)
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return ei

def neg_ei(x, gp, y_best, xi=0.001):
    return -expected_improvement(x, gp, y_best, xi)

# -------------------------
# Step 4: Optimize EI to get next candidate (propose location above)
# -------------------------


# -------------------------
# Step 5: Get next point (exploitative mode)
# -------------------------
y_best = y_all.max()
x_next = propose_location(gp, y_best, bounds, n_restarts=40, xi=0.001)

print("Next suggested point (exploit mode, safe bounds):", x_next)

x_next_6dp = np.round(x_next, 6)
x_next_6dp
print("Rounded to:", x_next_6dp)

# Week 5

In [ ]:
# assume previous additions have been executed
# Latest evaluated point (the one that gave 9.37782...) from Week 4
X_new = np.array([[0.122075, 0.010000, 0.420807, 0.010000, 0.839488, 0.990000, 0.010000, 0.990000]])
y_new = np.array([9.377829603931])

# -------------------------
# Step 1: Append new data
# -------------------------
X_all = np.vstack([X_all, X_new])
y_all = np.concatenate([y_all, y_new])

This new point produced a lower output (9.3778) than your current best (9.7668). It is not near the current best in input space (Euclidean distance ≈ 1.20) — in particular it flips several coordinates relative to the best (most notably dimension 8 and 6). So this evaluation is negative evidence for that region: it helps the GP learn the shape of the surface (it reduces uncertainty and lowers predicted mean where those coordinate changes occur). The practical consequence is: don’t switch strategy — append & refit the GP, then continue exploitative local refinement around the true best, while (optionally) doing a small amount of targeted exploration to test sensitive dimensions.

- Current best (so far): X_best ≈ [0.251068, 0.078758, 0.302102, 0.01, 0.99, 0.357561, 0.178631, 0.01] → y_best = 9.766762063933
- New point: X_new = [0.122075, 0.01, 0.420807, 0.01, 0.839488, 0.99, 0.01, 0.99] → y_new = 9.377829603931
- the biggest flips (dim 8 and dim 6) correlate with a drop in performance — a sign those coordinates are sensitive and likely important to the local optimum.

In [ ]:
# -------------------------
# Step 4: Optimize EI to get next candidate
# -------------------------
def propose_location(gp, y_best, bounds, n_restarts=40, xi=0.001):
    dim = bounds.shape[0]
    best_x = None
    best_val = 1e20

    # Random-start L-BFGS-B local optimization
    for _ in range(n_restarts):
        x0 = np.random.uniform(bounds[:, 0], bounds[:, 1])
        res = minimize(
            fun=neg_ei,
            x0=x0,
            args=(gp, y_best, xi),
            bounds=bounds,
            method='L-BFGS-B',
            options={'maxiter': 200}
        )
        if res.fun < best_val:
            best_val = res.fun
            best_x = res.x

    # Global backup using Differential Evolution
    try:
        de_res = differential_evolution(
            lambda x: neg_ei(x, gp, y_best, xi),
            bounds=bounds.tolist(),
            maxiter=200,
            polish=True,
            seed=0
        )
        if de_res.fun < best_val:
            best_val = de_res.fun
            best_x = de_res.x
    except Exception:
        pass

    # --------------------------------------
    # Soft clamp to avoid hitting exact edges
    # --------------------------------------
    epsilon = 0.01  # safety margin
    best_x = np.clip(best_x, epsilon, 1 - epsilon)

    return best_x

def expected_improvement(x, gp, y_best, xi=0.001):
    x = np.atleast_2d(x)
    mu, sigma = gp.predict(x, return_std=True)
    sigma = np.maximum(sigma, 1e-12)
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return ei

def neg_ei(x, gp, y_best, xi=0.001):
    return -expected_improvement(x, gp, y_best, xi)

In [ ]:
bounds = np.array([(0.0, 1.0)] * X_all.shape[1])
dim = X_all.shape[1]

kernel = Matern(length_scale=np.ones(dim), 
                length_scale_bounds=(1e-2, 1e2), 
                nu=2.5)

gp = GaussianProcessRegressor(kernel=kernel,
                              alpha=1e-6,
                              normalize_y=True,
                              n_restarts_optimizer=5,
                              random_state=0)
gp.fit(X_all, y_all)

y_best = y_all.max()
x_best = X_all[np.argmax(y_all)]

append point, refit GP, then propose next point using a local search around current best

In [ ]:
import numpy as np

# Build local bounds around x_best
local_radius = 0.12  # change 0.08-0.15 to tune locality
eps = 1e-3
dim = X_all.shape[1]
local_bounds = []
for i in range(dim):
    low = max(eps, x_best[i] - local_radius)
    high = min(1 - eps, x_best[i] + local_radius)
    # If a coordinate is already very tight (e.g. near 0.01/0.99), keep it tighter:
    if x_best[i] <= 0.02 or x_best[i] >= 0.98:
        # keep small wiggle
        low = max(eps, x_best[i] - 0.02)
        high = min(1 - eps, x_best[i] + 0.02)
    local_bounds.append((low, high))
local_bounds = np.array(local_bounds)

# Propose next point (exploitative EI)
x_next = propose_location(gp, y_best, local_bounds, n_restarts=20, xi=0.001)

print("x_best (current):", x_best)
print("y_best:", y_best)
print("Next candidate (local exploit):", x_next)


In [ ]:
x_next_6dp = np.round(x_next, 6)
x_next_6dp
print("Rounded to:", x_next_6dp)